TODO: 

I) version base
- (done)  reprendre classifieur de chat hiérarchique, remplacer encodeur seq2vec par un seq2seq (stacked lstm eg)
    cf HierarchicalChatSequenceClassification
- (done)  ajouter séquence de labels / comme si arbre ou forêt (possible ds graph_parser ?)

- (done) sousclasser graph_parser / modifs
    [pas la peine] plus facile en forcant label unique sur les arcs ? à voir
- (done/untested) brancher données réelles
      
      
/home/muller/anaconda3/envs/allennlp/lib/python3.7/site-packages/allennlp/data/dataset_readers/semantic_dependency_parsing.py


améliorations: 
- [done] déséquilibre class -> mettre poids dans la loss BCEwithlogitsloss 
- capacité modèle
- encodeur tour -> bert
- preprocessing des chats


In [1]:
import itertools
import logging
from typing import Any, Tuple, Dict, List, Iterable
import torch
import torch.optim as optim

In [2]:
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.data.fields import Field, LabelField, TextField, ListField, SequenceLabelField
from allennlp.data.instance import Instance
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token, Tokenizer, WordTokenizer, PretrainedTransformerTokenizer
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding

from allennlp.modules import Seq2VecEncoder, Seq2SeqEncoder
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.data.vocabulary import Vocabulary

from allennlp.training.trainer import Trainer
from allennlp.data.iterators import BucketIterator
from allennlp.common import Params

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
#from dummy_chat_reader import ChatReader
from irc_chat_reader import ChatReader

In [6]:
token_indexers = {"tokens": SingleIdTokenIndexer()}

tokenizer_cfg = Params({"word_splitter": {"language": "en"}})

tokenizer = Tokenizer.from_params(tokenizer_cfg)


reader = ChatReader(
    tokenizer=tokenizer,
    token_indexers=token_indexers,
    raw = True,
    #clip = 200
    )
train_instances = reader.read("../data/train")
dev_instances = reader.read("../data/dev")
vocab = Vocabulary.from_instances(train_instances+dev_instances)


0it [00:00, ?it/s]
2it [00:00, 13.79it/s]
4it [00:00, 15.16it/s]
6it [00:00, 14.83it/s]
8it [00:00, 14.32it/s]
11it [00:00, 16.24it/s]
13it [00:00, 16.58it/s]
15it [00:01, 12.74it/s]
17it [00:01, 13.13it/s]
19it [00:01, 13.75it/s]
21it [00:01, 10.81it/s]
23it [00:01,  9.98it/s]
25it [00:02,  9.20it/s]
27it [00:02,  8.93it/s]
28it [00:02,  9.19it/s]
29it [00:02,  8.95it/s]
31it [00:02,  9.16it/s]
33it [00:02, 10.35it/s]
35it [00:03, 11.20it/s]
37it [00:03,  9.68it/s]
39it [00:03,  8.84it/s]
40it [00:03,  8.76it/s]
42it [00:03,  9.78it/s]
44it [00:04,  9.27it/s]
46it [00:04, 10.23it/s]
48it [00:04, 10.54it/s]
50it [00:04, 10.99it/s]
53it [00:04, 12.12it/s]
55it [00:04, 10.70it/s]
57it [00:05,  8.60it/s]
59it [00:05,  7.55it/s]
60it [00:05,  7.70it/s]
62it [00:05,  8.71it/s]
63it [00:06,  8.73it/s]
64it [00:06,  8.77it/s]
65it [00:06,  8.48it/s]
66it [00:06,  8.49it/s]
67it [00:06,  8.37it/s]
68it [00:06,  8.30it/s]
70it [00:06,  9.24it/s]
72it [00:07,  9.33it/s]
74it [00:07,  9.56it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../data/dev/2016-12-19_20.raw.annotation.txt'

In [ ]:
#for i in train_instances:
#    #print(i)
#    i["lines"].index(vocab)
#    i["arcs"].index(vocab)
#    print(i["lines"].get_padding_lengths())

In [ ]:
i0 = train_instances[0]
print(i0)

Instance with fields:
 	 lines: ListField of 500 TextFields : 
 	 TextField of length 5 with text: 
 		[<, rolo, >, shazbotmcnasty, wait]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 TextField of length 15 with text: 
 		[<, DexterLB, >, still, no, effect, ,, home, folder, is, still, displayed, instead, of, ~/Desktop]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 TextField of length 14 with text: 
 		[<, acrobat, >, hi, i, m, new, to, ubuntu, can, smeone, hlepl, me, pls]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 TextField of length 19 with text: 
 		[<, acrobat, >, i, want, to, install, ubuntu, that, has, a, 64bit, compiler, on, a, 1, gb, usb,
		stick]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 TextField of length 92 with text: 
 		[<, arooni, -, mobile, _, _, _, >, hi, folks, ..., i, moved, my, karmic, desktop, from, one, side,
		of, the, room, to, the, other, ..., and, now, it, wo, nt, give, me, the, graphical, l

In [ ]:
c = 0
for n,i in enumerate(train_instances):
    nbarcs = len(i["arcs"].indices)
    if nbarcs==0: 
        c+=1 
        print(n)
print("zeros:",c)

zeros: 0


In [ ]:
#distance max
#for instance in train_instances:
#    maxl = max([abs(j-i) for (i,j) in instance["arcs"].indices])
#    print(maxl)
               

In [ ]:
turn_encoder_cfg = Params({"type":"gru",'input_size': 300, 'hidden_size': 100, 'num_layers': 1,
                  'dropout': 0.25, 'bidirectional': True
})
#can be changed dynamically encoder_cfg["type"] = "lstm"
# warning: if bidirectional, state output dimension is hidden_size x 2 -> model doesn't know that

turn_encoder = Seq2VecEncoder.from_params(turn_encoder_cfg)
turn_encoder.hidden_size = turn_encoder_cfg["hidden_size"]


chat_encoder_cfg = Params({"type":"gru",'input_size': 200, 'hidden_size': 100, 'num_layers': 2,
                  'dropout': 0.25, 'bidirectional': False
})
chat_encoder = Seq2SeqEncoder.from_params(chat_encoder_cfg)
chat_encoder.hidden_size = chat_encoder_cfg["hidden_size"]



glove_text_field_embedder = Embedding.from_params(vocab,Params({"pretrained_file": "https://s3-us-west-2.amazonaws.com/allennlp/datasets/glove/glove.6B.300d.txt.gz",
                                                          "embedding_dim": 300,
                                                          "trainable": False
}))

token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=300)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})



/home/muller/miniconda3/envs/allennlp2/lib/python3.7/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
400000it [00:07, 53679.01it/s]


In [ ]:
#chat_encoder

In [ ]:
# from biaffine parser, another config (not used)
chat_encoder_cfg =  Params({
            "type": "stacked_bidirectional_lstm",
            "hidden_size": 400,
            "input_size": 200,
            "num_layers": 3,
            "recurrent_dropout_probability": 0.3,
            "use_highway": True
        })

In [ ]:
from allennlp.models import Model
from typing import Dict, List, Iterable
from allennlp.modules import TimeDistributed
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits

In [ ]:
from allennlp.models import Model
from allennlp.common.checks import check_dimensions_match, ConfigurationError
from allennlp.data import Vocabulary
from allennlp.modules import Seq2SeqEncoder, TextFieldEmbedder, Embedding, InputVariationalDropout
from allennlp.modules.matrix_attention.bilinear_matrix_attention import BilinearMatrixAttention
from allennlp.modules import FeedForward
from allennlp.models.model import Model
from allennlp.nn import InitializerApplicator, Activation
#???? TODO from allennlp.nn.util import min_value_of_dtype -> only allennlp >= 1.0
def min_value_of_dtype(dtype: torch.dtype):
    """
    Returns the minimum value of a given PyTorch data type. Does not allow torch.bool.
    """
    return info_value_of_dtype(dtype).min
def info_value_of_dtype(dtype: torch.dtype):
    """
    Returns the `finfo` or `iinfo` object of a given PyTorch data type. Does not allow torch.bool.
    """
    if dtype == torch.bool:
        raise TypeError("Does not support torch.bool")
    elif dtype.is_floating_point:
        return torch.finfo(dtype)
    else:
        return torch.iinfo(dtype)


from allennlp.nn.util import get_text_field_mask
from allennlp.nn.util import get_lengths_from_binary_sequence_mask
from allennlp.training.metrics import F1Measure

import copy
from overrides import overrides
import torch
from torch.nn.modules import Dropout
import numpy

class ChatGraphParser(Model):
    """
    A Parser for arbitrary graph structures.

    Registered as a `Model` with name "graph_parser".

    # Parameters

    vocab : `Vocabulary`, required
        A Vocabulary, required in order to compute sizes for input/output projections.
    text_field_embedder : `TextFieldEmbedder`, required
        Used to embed the `tokens` `TextField` we get as input to the model.
    encoder : `Seq2SeqEncoder`
        The encoder (with its own internal stacking) that we will use to generate representations
        of tokens.
    tag_representation_dim : `int`, required.
        The dimension of the MLPs used for arc tag prediction.
    arc_representation_dim : `int`, required.
        The dimension of the MLPs used for arc prediction.
    tag_feedforward : `FeedForward`, optional, (default = None).
        The feedforward network used to produce tag representations.
        By default, a 1 layer feedforward network with an elu activation is used.
    arc_feedforward : `FeedForward`, optional, (default = None).
        The feedforward network used to produce arc representations.
        By default, a 1 layer feedforward network with an elu activation is used.
    pos_tag_embedding : `Embedding`, optional.
        Used to embed the `pos_tags` `SequenceLabelField` we get as input to the model.
    dropout : `float`, optional, (default = 0.0)
        The variational dropout applied to the output of the encoder and MLP layers.
    input_dropout : `float`, optional, (default = 0.0)
        The dropout applied to the embedded text input.
    edge_prediction_threshold : `int`, optional (default = 0.5)
        The probability at which to consider a scored edge to be 'present'
        in the decoded graph. Must be between 0 and 1.
    initializer : `InitializerApplicator`, optional (default=`InitializerApplicator()`)
        Used to initialize the model parameters.
    """

    def __init__(
        self,
        vocab: Vocabulary,
        text_field_embedder: TextFieldEmbedder,
        turn_encoder: Seq2VecEncoder, 
        chat_encoder: Seq2SeqEncoder,
        arc_representation_dim: int,
        arc_feedforward: FeedForward = None,
        dropout: float = 0.0,
        input_dropout: float = 0.0,
        edge_prediction_threshold: float = 0.5,
        positive_class_weight = 40,
        prediction_window = 13, # dont predict edge further apart
        initializer: InitializerApplicator = InitializerApplicator(),
        debug = False,
        **kwargs,
    ) -> None:
        super().__init__(vocab, **kwargs)
        
        self.text_field_embedder = text_field_embedder
        self.turn_encoder = TimeDistributed(turn_encoder)
        self.chat_encoder = chat_encoder
        
        self.edge_prediction_threshold = edge_prediction_threshold
        if not 0 < edge_prediction_threshold < 1:
            raise ConfigurationError(
                f"edge_prediction_threshold must be between "
                f"0 and 1 (exclusive) but found {edge_prediction_threshold}."
            )

        encoder_dim = chat_encoder.get_output_dim()

        self.head_arc_feedforward = arc_feedforward or FeedForward(
            encoder_dim, 1, arc_representation_dim, Activation.by_name("elu")()
        )
        self.child_arc_feedforward = copy.deepcopy(self.head_arc_feedforward)

        self.arc_attention = BilinearMatrixAttention(
            arc_representation_dim, arc_representation_dim, use_input_biases=True
        )

        self._dropout = InputVariationalDropout(dropout)
        self._input_dropout = Dropout(input_dropout)

        representation_dim = turn_encoder.get_output_dim()

        self._unlabelled_f1 = F1Measure(positive_label=1)
        #  with weight favoring recall of positive class 
        self._arc_loss = torch.nn.BCEWithLogitsLoss(reduction="none",
                                                    pos_weight=torch.tensor([positive_class_weight]))
        
        self.prediction_window = prediction_window
        initializer(self)
        # useful for debugging
        self.iter_count = 0 
        self.debug = debug
    # init done
        
    # todo 
    @overrides
    def forward(
        self,  # type: ignore
        lines,
        arcs: torch.LongTensor = None,
        metadata: List[Dict[str, Any]] = None,
    ) -> Dict[str, torch.Tensor]:

        """
        # Parameters

        lines: the chat as a list of turns, each being a list of token
        TODO: add metadata to instances
        metadata : List[Dict[str, Any]], optional (default = None)
            A dictionary of metadata for each batch element which has keys:
                tokens : `List[str]`, required.
                    The original string tokens in the sentence.
        arcs : a tensor containing the adjacency matrix for the instance dependencies between turns
            Has shape `(batch_size, sequence_length, sequence_length)`.

        # Returns

        An output dictionary.
        """
        #########
        # this is the part where chat is encoded as sequence of turn encodings
        #########
        # mask for each turn of each chat of the batch: shape = (batch_size x max_turns x tokens)
        token_mask = get_text_field_mask(lines,num_wrapping_dims=1)

        # chat turns fetching embedding
        # turns_embedding tensor is (batch_size x turns x max tokens x token embedding size)
        turns_embeddings = self.text_field_embedder(lines,num_wrapping_dims=1)
      
        # encoding turns
        # turn_h has shape (batch_size x turns x encoder_output_size) 
        turn_h = self.turn_encoder(turns_embeddings,token_mask)
        
        # mask for chats is now nb of turns; beware weird return type of torch.max (tuple) 
        chat_mask = token_mask.max(axis=2)[0]
        
        # renaming to mask -> easier to transpose the rest of graph_parser
        mask = chat_mask
        
        # graph parser goes on
        # leave input dropout for now
        # embedded_text_input = turn_h equivalent in hierarchical sequence -> renaming 
        #embedded_text_input = self._input_dropout(embedded_text_input)
        embedded_text_input = turn_h
        # we keep graph parser original name for now
        # encoded_text = encoded chat = self.chat_encoder(turn_h,chat_mask) equivalent in hierarchical sequence
        encoded_text = self.chat_encoder(embedded_text_input, mask)

        encoded_text = self._dropout(encoded_text)

        # shape (batch_size, sequence_length, arc_representation_dim)
        head_arc_representation = self._dropout(self.head_arc_feedforward(encoded_text))
        child_arc_representation = self._dropout(self.child_arc_feedforward(encoded_text))

        # shape (batch_size, sequence_length, tag_representation_dim)
        #head_tag_representation = self._dropout(self.head_tag_feedforward(encoded_text))
        #child_tag_representation = self._dropout(self.child_tag_feedforward(encoded_text))
        
        # shape (batch_size, sequence_length, sequence_length)
        arc_scores = self.arc_attention(head_arc_representation, child_arc_representation)
        
        # shape (batch_size, num_tags, sequence_length, sequence_length)
        #arc_tag_logits = self.tag_bilinear(head_tag_representation, child_tag_representation)
        # Switch to (batch_size, sequence_length, sequence_length, num_tags)
        #arc_tag_logits = arc_tag_logits.permute(0, 2, 3, 1).contiguous()

        # Since we'll be doing some additions, using the min value will cause underflow
        # CHAT: unncessary since we dont have a loss for labels
        #minus_mask = ~mask * min_value_of_dtype(arc_scores.dtype) / 10
        #arc_scores = arc_scores + minus_mask.unsqueeze(2) + minus_mask.unsqueeze(1)

        self.iter_count += 1
        # Debugging every 10 epochs
        if self.debug and self.iter_count%(153*2)==0:
            breakpoint()
            
        arc_probs = self._greedy_decode(arc_scores, mask,self.prediction_window)

        output_dict = {"arc_probs": arc_probs, "mask": mask}

        if metadata:
            output_dict["tokens"] = [meta["tokens"] for meta in metadata]

        arc_tags = arcs # gold labels -> here just the adjacency matrix 0/1 ? 
        if arc_tags is not None:
            arc_nll= self._construct_loss(
                arc_scores=arc_scores, arc_tags=arc_tags, mask=mask
            )
            # same here with no arc relations ; keep all anyway to prevent ubgs downstream (TODO: coherent renaming)
            output_dict["loss"] = arc_nll 
            output_dict["arc_loss"] = arc_nll
            

            # Make the arc tags not have negative values anywhere
            # (by default, no edge is indicated with -1).
            # NB re chat: probably not useful, but kept as a precaution
            arc_indices = (arc_tags != -1).float()
            tag_mask = mask.unsqueeze(1) & mask.unsqueeze(2)
            one_minus_arc_probs = 1 - arc_probs
            # We stack scores here because the f1 measure expects a
            # distribution, rather than a single value.
            self._unlabelled_f1(
                torch.stack([one_minus_arc_probs, arc_probs], -1), arc_indices, tag_mask
            )
        
        
        return output_dict
    # modified / untested
    
    def _construct_loss(
        self,
        arc_scores: torch.Tensor,
        arc_tags: torch.Tensor,
        mask: torch.BoolTensor,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Computes the arc and tag loss for an adjacency matrix.

        # Parameters

        arc_scores : `torch.Tensor`, required.
            A tensor of shape (batch_size, sequence_length, sequence_length) used to generate a
            binary classification decision for whether an edge is present between two words.
        #####arc_tag_logits : `torch.Tensor`, required.
        #####    A tensor of shape (batch_size, sequence_length, sequence_length, num_tags) used to generate
        #####    a distribution over edge tags for a given edge.
        arc_tags : `torch.Tensor`, required.
            A tensor of shape (batch_size, sequence_length, sequence_length).
            The labels for every arc.
        mask : `torch.BoolTensor`, required.
            A mask of shape (batch_size, sequence_length), denoting unpadded
            elements in the sequence.

        # Returns

        arc_nll : `torch.Tensor`, required.
            The negative log likelihood from the arc loss.
        tag_nll : `torch.Tensor`, required.
            The negative log likelihood from the arc tag loss.
        """
        arc_indices = (arc_tags != -1).float()
        # Make the arc tags not have negative values anywhere
        # (by default, no edge is indicated with -1).
        arc_tags = arc_tags * arc_indices
        arc_nll = self._arc_loss(arc_scores, arc_indices) * mask.unsqueeze(1) * mask.unsqueeze(2)
        # We want the mask for the tags to only include the unmasked words
        # and we only care about the loss with respect to the gold arcs.
        tag_mask = mask.unsqueeze(1) * mask.unsqueeze(2) * arc_indices

        #batch_size, sequence_length, _, num_tags = arc_tag_logits.size()
        #original_shape = [batch_size, sequence_length, sequence_length]
        #reshaped_logits = arc_tag_logits.view(-1, num_tags)
        reshaped_tags = arc_tags.view(-1)
        #tag_nll = (
        #    self._tag_loss(reshaped_logits, reshaped_tags.long()).view(original_shape) * tag_mask
        #)

        valid_positions = tag_mask.sum()

        arc_nll = arc_nll.sum() / valid_positions.float()
        #tag_nll = tag_nll.sum() / valid_positions.float()
        return arc_nll#, tag_nll
    # modified/untested

    @staticmethod
    def _greedy_decode(
        arc_scores: torch.Tensor, 
        mask: torch.BoolTensor,
        prediction_window: int,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Decodes the head and head tag predictions by decoding the unlabeled arcs
        independently for each word and then again, predicting the head tags of
        these greedily chosen arcs independently.

        # Parameters

        arc_scores : `torch.Tensor`, required.
            A tensor of shape (batch_size, sequence_length, sequence_length) used to generate
            a distribution over attachments of a given word to all other words.
        ###arc_tag_logits : `torch.Tensor`, required.
        ###    A tensor of shape (batch_size, sequence_length, sequence_length, num_tags) used to
        ###    generate a distribution over tags for each arc.
        mask : `torch.BoolTensor`, required.
            A mask of shape (batch_size, sequence_length).

        # Returns

        arc_probs : `torch.Tensor`
            A tensor of shape (batch_size, sequence_length, sequence_length) representing the
            probability of an arc being present for this edge.
        ####arc_tag_probs : `torch.Tensor`
        ####    A tensor of shape (batch_size, sequence_length, sequence_length, sequence_length)
        ####    representing the distribution over edge tags for a given edge.
        """
        # Mask the diagonal, because we don't self edges.
        # WARNING: might not be the case for chats ? -> should be an option
        inf_diagonal_mask = torch.diag(arc_scores.new(mask.size(1)).fill_(-numpy.inf))
        # no edges going backwards
        triangle_upper_mask = torch.triu(arc_scores.new(mask.size(1),mask.size(1)).fill_(-numpy.inf))
        # prevent edges between turns more than CONSTANT turns appart
        up = 1-torch.triu(arc_scores.new(mask.size(1),mask.size(1)).fill_(1),diagonal=prediction_window)
        down = 1-torch.tril(arc_scores.new(mask.size(1),mask.size(1)).fill_(1),diagonal=-prediction_window)
        diag_mask = numpy.log(((up+down)-1))
        # up_mask = torch.triu(arc_scores.new_zeros(mask.size(1),mask.size(1),diagonal=-15)
        # down_mask = torch.tril(arc_scores.new_zeros(mask.size(1),mask.size(1),diagonal=15)
        # away_mask = (up_mask == down_mask)
        arc_scores = arc_scores + inf_diagonal_mask + diag_mask + triangle_upper_mask
        # shape (batch_size, sequence_length, sequence_length, num_tags)
        #arc_tag_logits = arc_tag_logits + inf_diagonal_mask.unsqueeze(0).unsqueeze(-1)
        # Mask padded tokens, because we only want to consider actual word -> word edges.
        # CHAT: this is the wrong torch version lol this does not work/ confusion int/bools
        # minus_mask = ~mask.unsqueeze(2)
        # CHAT: this should work with torch>1.4
        #minus_mask = (mask<1).unsqueeze(2)
        minus_mask = (mask.unsqueeze(1) & mask.unsqueeze(2))<1
        
        arc_scores.masked_fill_(minus_mask, -numpy.inf)
        # 
        
        #arc_tag_logits.masked_fill_(minus_mask.unsqueeze(-1), -numpy.inf)
        # shape (batch_size, sequence_length, sequence_length)
        arc_probs = arc_scores.sigmoid()
        # shape (batch_size, sequence_length, sequence_length, num_tags)
        #arc_tag_probs = torch.nn.functional.softmax(arc_tag_logits, dim=-1)
        return arc_probs#, arc_tag_probs
    # modified / untested
    
    @overrides
    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        metrics = {}
        precision, recall, f1_measure = self._unlabelled_f1.get_metric(reset)
        metrics["precision"] = precision
        metrics["recall"] = recall
        metrics["f1"] = f1_measure
        return metrics

 

In [ ]:
#model = HierarchicalChatSequenceClassification(vocab,word_embeddings,turn_encoder,chat_encoder)

if False:
    reader = ChatReader(
        tokenizer=tokenizer,
        token_indexers=token_indexers,
        raw = True,
        #sample = 50, 
        #clip = 200
        )
    train_instances = reader.read("../data/train")
    vocab = Vocabulary.from_instances(train_instances)


arc_representation_dim = 50 
model = ChatGraphParser(vocab,word_embeddings,
                        turn_encoder,chat_encoder,arc_representation_dim,
                        prediction_window = 2,
                        positive_class_weight = 1000,
                        debug=True,
                        edge_prediction_threshold=0.5)


from allennlp.training.optimizers import Optimizer 
trainer_cfg = Params({
        "cuda_device": 1,
        "grad_norm": 5,
        "num_epochs": 100,
        "optimizer": {
            "type": "dense_sparse_adam",
            "betas": [
                0.9,
                0.9
            ]
        },
        "patience": 50,
})
opt_cfg = trainer_cfg.pop("optimizer")

In [ ]:


#optimizer = optim.SGD(model.parameters(), lr=0.1)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# not tested
#optimizer = Optimizer.from_params(model_parameters=model.parameters(),
#                                  params=opt_cfg)

iterator = BucketIterator(batch_size=4,
                          sorting_keys=[("lines","list_num_tokens")])
iterator.index_with(vocab)



trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  num_epochs=100,
                  grad_norm=5,
                  patience=10,
                  cuda_device = -1,
                  train_dataset=train_instances,
                  validation_dataset=dev_instances,
                  should_log_parameter_statistics = False
)

In [ ]:
trainer.train()

  0%|          | 0/39 [00:00<?, ?it/s]/home/muller/miniconda3/envs/allennlp2/lib/python3.7/site-packages/ipykernel_launcher.py:342: RuntimeWarning: divide by zero encountered in log
precision: 0.1860, recall: 0.1742, f1: 0.1799, loss: 958.1749 ||: 100%|██████████| 39/39 [02:06<00:00,  2.28s/it] 
  0%|          | 0/39 [00:00<?, ?it/s]/home/muller/miniconda3/envs/allennlp2/lib/python3.7/site-packages/ipykernel_launcher.py:342: RuntimeWarning: divide by zero encountered in log
precision: 0.1861, recall: 0.1824, f1: 0.1842, loss: 935.6593 ||: 100%|██████████| 39/39 [02:01<00:00,  3.14s/it]
  0%|          | 0/39 [00:00<?, ?it/s]/home/muller/miniconda3/envs/allennlp2/lib/python3.7/site-packages/ipykernel_launcher.py:342: RuntimeWarning: divide by zero encountered in log
precision: 0.1861, recall: 0.1824, f1: 0.1842, loss: 931.8195 ||: 100%|██████████| 39/39 [02:01<00:00,  2.74s/it]
  0%|          | 0/39 [00:00<?, ?it/s]/home/muller/miniconda3/envs/allennlp2/lib/python3.7/site-packages/ipyker

> <ipython-input-14-6a0c67557269>(215)forward()
-> arc_probs = self._greedy_decode(arc_scores, mask,self.prediction_window)
(Pdb) arc_probs
*** NameError: name 'arc_probs' is not defined
(Pdb) next
> <ipython-input-14-6a0c67557269>(217)forward()
-> output_dict = {"arc_probs": arc_probs, "mask": mask}
(Pdb) arc_probs
tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.6122, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.6327, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.6232, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.6430, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
  

precision: 0.1865, recall: 0.1823, f1: 0.1844, loss: 913.5578 ||: 100%|██████████| 39/39 [07:44<00:00, 14.96s/it] 
  0%|          | 0/39 [00:00<?, ?it/s]/home/muller/miniconda3/envs/allennlp2/lib/python3.7/site-packages/ipykernel_launcher.py:342: RuntimeWarning: divide by zero encountered in log
precision: 0.1880, recall: 0.1818, f1: 0.1848, loss: 884.4699 ||: 100%|██████████| 39/39 [01:59<00:00,  4.24s/it]
  0%|          | 0/39 [00:00<?, ?it/s]/home/muller/miniconda3/envs/allennlp2/lib/python3.7/site-packages/ipykernel_launcher.py:342: RuntimeWarning: divide by zero encountered in log
precision: 0.1873, recall: 0.1820, f1: 0.1846, loss: 861.3255 ||: 100%|██████████| 39/39 [02:01<00:00,  4.17s/it]
  0%|          | 0/39 [00:00<?, ?it/s]/home/muller/miniconda3/envs/allennlp2/lib/python3.7/site-packages/ipykernel_launcher.py:342: RuntimeWarning: divide by zero encountered in log
precision: 0.1887, recall: 0.1815, f1: 0.1850, loss: 826.0029 ||: 100%|██████████| 39/39 [02:00<00:00,  2.57s/i

> <ipython-input-14-6a0c67557269>(215)forward()
-> arc_probs = self._greedy_decode(arc_scores, mask,self.prediction_window)
(Pdb) self.debug=False
(Pdb) continue


precision: 0.1865, recall: 0.1822, f1: 0.1843, loss: 542.6287 ||: 100%|██████████| 39/39 [10:51<00:00,  5.20s/it] 
  0%|          | 0/39 [00:00<?, ?it/s]/home/muller/miniconda3/envs/allennlp2/lib/python3.7/site-packages/ipykernel_launcher.py:342: RuntimeWarning: divide by zero encountered in log
precision: 0.1866, recall: 0.1822, f1: 0.1844, loss: 507.5445 ||: 100%|██████████| 39/39 [01:55<00:00,  3.25s/it]
  0%|          | 0/39 [00:00<?, ?it/s]/home/muller/miniconda3/envs/allennlp2/lib/python3.7/site-packages/ipykernel_launcher.py:342: RuntimeWarning: divide by zero encountered in log
precision: 0.1864, recall: 0.1821, f1: 0.1842, loss: 468.3121 ||: 100%|██████████| 39/39 [01:58<00:00,  3.45s/it]
  0%|          | 0/39 [00:00<?, ?it/s]/home/muller/miniconda3/envs/allennlp2/lib/python3.7/site-packages/ipykernel_launcher.py:342: RuntimeWarning: divide by zero encountered in log
precision: 0.1862, recall: 0.1819, f1: 0.1840, loss: 432.5802 ||: 100%|██████████| 39/39 [01:57<00:00,  3.42s/i

precision: 0.1862, recall: 0.1823, f1: 0.1842, loss: 246.6451 ||: 100%|██████████| 39/39 [02:01<00:00,  2.43s/it]
  0%|          | 0/39 [00:00<?, ?it/s]/home/muller/miniconda3/envs/allennlp2/lib/python3.7/site-packages/ipykernel_launcher.py:342: RuntimeWarning: divide by zero encountered in log
precision: 0.1862, recall: 0.1823, f1: 0.1842, loss: 238.9331 ||: 100%|██████████| 39/39 [01:56<00:00,  3.18s/it]
  0%|          | 0/39 [00:00<?, ?it/s]/home/muller/miniconda3/envs/allennlp2/lib/python3.7/site-packages/ipykernel_launcher.py:342: RuntimeWarning: divide by zero encountered in log
precision: 0.1862, recall: 0.1823, f1: 0.1842, loss: 239.6788 ||: 100%|██████████| 39/39 [01:57<00:00,  2.53s/it]
  0%|          | 0/39 [00:00<?, ?it/s]/home/muller/miniconda3/envs/allennlp2/lib/python3.7/site-packages/ipykernel_launcher.py:342: RuntimeWarning: divide by zero encountered in log
precision: 0.1862, recall: 0.1823, f1: 0.1842, loss: 236.3571 ||: 100%|██████████| 39/39 [01:58<00:00,  2.92s/it

In [ ]:
opt_cfg

In [ ]:
torch.tensor([0.1,0.99])

In [ ]:
mask = torch.tensor([[1, 1, 1, 0],
        [1, 1, 1, 1]])

In [ ]:
inf = numpy.inf 
arc_scores = torch.tensor([[[  -inf, 0.1252, 0.1161, 0.0475],
         [0.0977,   -inf, 0.1015, 0.0373],
         [0.1541, 0.1673,   -inf, 0.0934],
         [0.0563, 0.0730, 0.0623,   -inf]],

        [[  -inf, 0.0947, 0.1048, 0.1114],
         [0.1385,   -inf, 0.1681, 0.1755],
         [0.1114, 0.1310,   -inf, 0.1448],
         [0.1274, 0.1454, 0.1537,   -inf]]])

In [ ]:
minus_mask = (mask<1).unsqueeze(2)    
arc_scores.masked_fill_(minus_mask, -numpy.inf)         

In [ ]:
arc_probs = arc_scores.sigmoid()    

In [ ]:
mask

In [ ]:
minus_mask = (mask.unsqueeze(1) & mask.unsqueeze(2))<1

In [ ]:
minus_mask

In [ ]:
arc_scores.masked_fill_(minus_mask, -numpy.inf)         

In [ ]:
(mask<1).unsqueeze(1)